# CellProfiler features for the supervised baseline experiment

In [1]:
import os
import pandas as pd
import numpy as np

Function for extracting metadata and CellProfiler feature column indices:

In [2]:
def get_feature_cols(df: pd.DataFrame, 
                     features_type: str="cellprofiler") -> list() :
    """Splits columns of input dataframe into columns contining metadata and columns containing morphological profile features
    :param df: input data frame
    :param features_type:
        "standard" for features named as feature_1, feature_2 ..
        "CellProfiler": for cell-centric Cell Profiler features names as Cells_ , Nuclei_ , Cytoplasm_
    :return : feature_columns , info_columns
    """
    if features_type.lower() =="cellprofiler":
        feature_cols = [ c for c in df.columns if ( c.startswith("Cells_") | c.startswith("Nuclei_") | c.startswith("Cytoplasm_" )) & ("Metadata" not in c) ]
    elif features_type=="standard":
        feature_cols = [ c for c in df.columns if ( c.startswith("feature_") | c.startswith('emb') ) ]
    else:
        raise NotImplementedError("Feature Type not implemented. Options: CellProfiler, standard")
    info_cols =  list( set(df.columns).difference(set(feature_cols)) )
    return feature_cols, info_cols

In [3]:
def get_feature_cols_with_key(df: pd.DataFrame, 
                     features_type: str="cellprofiler") -> list() :
    """Splits columns of input dataframe into columns contining metadata and columns containing morphological profile features
    :param df: input data frame
    :param features_type:
        "standard" for features named as feature_1, feature_2 ..
        "CellProfiler": for cell-centric Cell Profiler features names as Cells_ , Nuclei_ , Cytoplasm_
    :return : feature_columns , info_columns
    """
    if features_type.lower() =="cellprofiler":
        feature_cols = [ c for c in df.columns if ( c.startswith("Cells_") | c.startswith("Nuclei_") | c.startswith("Cytoplasm_" )| c.startswith("INCHIKEY" )) & ("Metadata" not in c) ]
    elif features_type=="standard":
        feature_cols = [ c for c in df.columns if ( c.startswith("feature_") | c.startswith('emb') ) ]
    else:
        raise NotImplementedError("Feature Type not implemented. Options: CellProfiler, standard")
    info_cols =  list( set(df.columns).difference(set(feature_cols)) )
    return feature_cols, info_cols

Function for impute Nan values

In [4]:
def impute_features( data, feature_cols, outlier_threshold=1000):
    
    for feat in feature_cols:
        impute_idx = data[feat].isin([np.inf, -np.inf, np.nan])
        if impute_idx.sum()>0:
            print('Nan or inf feature %s for %i samples'%(feat, impute_idx.sum()))
        if outlier_threshold:
            impute_idx = impute_idx | (data[feat].abs()>outlier_threshold)
        if impute_idx.sum()>0:
            data.loc[ impute_idx, feat] = data.loc[ impute_idx==False, feat].median()
            print('Imputing to median %s for %i samples'%(feat, impute_idx.sum()))
    return data

Load CellProfiler features:

In [5]:
cellprof_df = pd.read_csv('/SSL_data/supervisedCNNs/Bray_bioactives/embeddings/CellProfiler/well_features.csv')

/wuppertal/gpcsx/projects/supervised_baseline/ipykernel_1319577/432750063.py:1: DtypeWarning: Columns (7,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  cellprof_df = pd.read_csv('/raid/cache/HCA_data_lake/Bray_bioactives/embeddings/CellProfiler/well_features.csv')


In [6]:
cellprof_df.shape

(47139, 821)

In [7]:
cellprof_df.isnull().values.any()
cellprof_df.isnull().sum().sum()

192274

In [8]:
feature_cols, meta_cols = get_feature_cols(cellprof_df)

In [9]:
data_impute_df = impute_features( cellprof_df.copy(), feature_cols)

Nan or inf feature Cells_AreaShape_FormFactor for 3 samples
Imputing to median Cells_AreaShape_FormFactor for 3 samples
Nan or inf feature Cells_Correlation_Manders_AGP_ER for 145 samples
Imputing to median Cells_Correlation_Manders_AGP_ER for 145 samples
Nan or inf feature Cells_Correlation_Manders_DNA_AGP for 1024 samples
Imputing to median Cells_Correlation_Manders_DNA_AGP for 1030 samples
Nan or inf feature Cells_Correlation_Manders_DNA_ER for 145 samples
Imputing to median Cells_Correlation_Manders_DNA_ER for 145 samples
Imputing to median Cells_Correlation_Manders_ER_AGP for 3 samples
Nan or inf feature Cells_Correlation_Manders_ER_DNA for 26 samples
Imputing to median Cells_Correlation_Manders_ER_DNA for 26 samples
Nan or inf feature Cells_Correlation_Manders_RNA_ER for 137 samples
Imputing to median Cells_Correlation_Manders_RNA_ER for 137 samples
Nan or inf feature Cells_Neighbors_AngleBetweenNeighbors_Adjacent for 4 samples
Imputing to median Cells_Neighbors_AngleBetweenNeigh

In [10]:
data_impute_df.head

<bound method NDFrame.head of                           INCHIKEY  Plate Well            broad_sample  \
0      RIHXBZGHHXCGPP-MZKRTTBSSA-N  26795  A10  BRD-K53839527-001-01-4   
1      RIHXBZGHHXCGPP-MZKRTTBSSA-N  26679  A10  BRD-K53839527-001-01-4   
2      RIHXBZGHHXCGPP-MZKRTTBSSA-N  26680  A10  BRD-K53839527-001-01-4   
3      RIHXBZGHHXCGPP-MZKRTTBSSA-N  26794  A10  BRD-K53839527-001-01-4   
4      RIHXBZGHHXCGPP-SXWKCWPCSA-N  26795  A12  BRD-K71719006-001-01-6   
...                            ...    ...  ...                     ...   
47134  GTYKJOKWQTUGAM-FVBCXUTKSA-N  25862  M07  BRD-K79908599-001-01-0   
47135  GTYKJOKWQTUGAM-FOUYOVOOSA-N  25885  M11  BRD-K45134469-001-01-1   
47136  GTYKJOKWQTUGAM-FOUYOVOOSA-N  25858  M11  BRD-K45134469-001-01-1   
47137  GTYKJOKWQTUGAM-FOUYOVOOSA-N  25859  M11  BRD-K45134469-001-01-1   
47138  GTYKJOKWQTUGAM-FOUYOVOOSA-N  25862  M11  BRD-K45134469-001-01-1   

      pert_type  Site_Count  Count_Cytoplasm chemist_name  Count_Cells  \
0      

Check how the data looks like

In [11]:
data_impute_df.shape

(47139, 821)

In [12]:
feature_cols, meta_cols = get_feature_cols_with_key(data_impute_df)

In [13]:
print(f"Number of CellProfiler features: {len(feature_cols)}")

Number of CellProfiler features: 632


In [14]:
print(feature_cols)

['INCHIKEY', 'Cells_AreaShape_BoundingBoxArea', 'Cells_AreaShape_BoundingBoxMaximum_X', 'Cells_AreaShape_Compactness', 'Cells_AreaShape_Eccentricity', 'Cells_AreaShape_EulerNumber', 'Cells_AreaShape_Extent', 'Cells_AreaShape_FormFactor', 'Cells_AreaShape_Orientation', 'Cells_AreaShape_Solidity', 'Cells_AreaShape_Zernike_1_1', 'Cells_AreaShape_Zernike_2_0', 'Cells_AreaShape_Zernike_2_2', 'Cells_AreaShape_Zernike_3_1', 'Cells_AreaShape_Zernike_3_3', 'Cells_AreaShape_Zernike_4_0', 'Cells_AreaShape_Zernike_4_2', 'Cells_AreaShape_Zernike_4_4', 'Cells_AreaShape_Zernike_5_1', 'Cells_AreaShape_Zernike_5_3', 'Cells_AreaShape_Zernike_5_5', 'Cells_AreaShape_Zernike_6_0', 'Cells_AreaShape_Zernike_6_2', 'Cells_AreaShape_Zernike_6_6', 'Cells_AreaShape_Zernike_7_1', 'Cells_AreaShape_Zernike_7_3', 'Cells_AreaShape_Zernike_7_7', 'Cells_AreaShape_Zernike_8_0', 'Cells_AreaShape_Zernike_8_2', 'Cells_AreaShape_Zernike_8_4', 'Cells_AreaShape_Zernike_8_8', 'Cells_AreaShape_Zernike_9_1', 'Cells_AreaShape_Zern

## Datasplits
The datasplits from the Hofmarcher et al study are in the following directory:

In [15]:
datasplit_dir = '/SSL_data/supervisedCNNs/Bray_bioactives/datasplits_Hofmarcher'

There were 3 versions of train/val/test splits:

In [16]:
sorted(os.listdir(datasplit_dir))

['datasplit1-test.csv',
 'datasplit1-train.csv',
 'datasplit1-val.csv',
 'datasplit2-test.csv',
 'datasplit2-train.csv',
 'datasplit2-val.csv',
 'datasplit3-test.csv',
 'datasplit3-train.csv',
 'datasplit3-val.csv']

To obtain train splits, load e.g. `datasplit1-train.csv` and use `INCHIKEY` column to subset the feature and activity data:

In [17]:
train_df_3 = pd.read_csv(os.path.join(datasplit_dir, 'datasplit3-train.csv'))

In [18]:
train_df_3[['INCHIKEY']].head()

,INCHIKEY
0,CAJIGINSTLKQMM-UHFFFAOYSA-N
1,CAJIGINSTLKQMM-UHFFFAOYSA-N
2,CAJIGINSTLKQMM-UHFFFAOYSA-N
3,CAJIGINSTLKQMM-UHFFFAOYSA-N
4,CAJIGINSTLKQMM-UHFFFAOYSA-N


In [19]:
train_df_3.shape

(198737, 17)

In [20]:
test_df_3 = pd.read_csv(os.path.join(datasplit_dir, 'datasplit3-test.csv'))
test_df_3.shape

(56687, 17)

In [21]:
val_df_3 = pd.read_csv(os.path.join(datasplit_dir, 'datasplit3-val.csv'))
val_df_3.shape

(28610, 17)

check the labels 

In [22]:
bioactivity_labels_dir = '/SSL_data/supervisedCNNs/Bray_bioactives/ChEMBL/Label_Matrix_Hofmarcher'

In [23]:
compound_index_df = pd.read_csv(os.path.join(bioactivity_labels_dir, 'row-compound-index.csv'))

In [24]:
compound_index_df.head()

,INDEX,INCHIKEY
0,0,IENZQIKPVFGBNW-UHFFFAOYSA-N
1,1,GSDSWSVVBLHKDQ-UHFFFAOYSA-N
2,2,CGIGDMFJXJATDK-UHFFFAOYSA-N
3,3,DSXXEELGXBCYNQ-UHFFFAOYSA-N
4,4,MYSWGUAQZAJSOK-UHFFFAOYSA-N


In [25]:
print(f"Number of compound indexes: {len(compound_index_df)}")

Number of compound indexes: 10574


Now we will merge compound with featuers

In [26]:
data_impute_df = data_impute_df[feature_cols]
compound_index_featuers = pd.merge(compound_index_df, data_impute_df, on='INCHIKEY')
print(compound_index_featuers.head())

   INDEX                     INCHIKEY  Cells_AreaShape_BoundingBoxArea  \
0      0  IENZQIKPVFGBNW-UHFFFAOYSA-N                         -0.92419   
1      0  IENZQIKPVFGBNW-UHFFFAOYSA-N                          6.47680   
2      0  IENZQIKPVFGBNW-UHFFFAOYSA-N                          0.61509   
3      0  IENZQIKPVFGBNW-UHFFFAOYSA-N                          0.13455   
4      0  IENZQIKPVFGBNW-UHFFFAOYSA-N                          4.76600   

   Cells_AreaShape_BoundingBoxMaximum_X  Cells_AreaShape_Compactness  \
0                              -0.72736                     -0.13691   
1                               1.29660                      5.07890   
2                               0.62653                      0.56783   
3                              -0.15419                      0.22563   
4                               2.99650                      4.03550   

   Cells_AreaShape_Eccentricity  Cells_AreaShape_EulerNumber  \
0                     -1.138300                     -0.407

In [27]:
assay_index_df = pd.read_csv(os.path.join(bioactivity_labels_dir, 'column-assay-index.csv'))

In [28]:
assay_index_df.head()

,ASSAY_ID
0,600885
1,688422
2,688493
3,688810
4,688812


In [29]:
from scipy.io import mmread
the_matrix = mmread("/SSL_data/supervisedCNNs/Bray_bioactives/ChEMBL/Label_Matrix_Hofmarcher/label-matrix.mtx")
print(the_matrix.shape)

(10574, 209)


In [30]:
print(the_matrix)

  (6537, 0)	1
  (6542, 0)	1
  (6543, 0)	1
  (6552, 0)	-1
  (6689, 0)	-1
  (6708, 0)	1
  (6821, 0)	1
  (6829, 0)	1
  (8169, 0)	1
  (8170, 0)	1
  (8174, 0)	1
  (8175, 0)	1
  (8176, 0)	1
  (8177, 0)	1
  (8180, 0)	1
  (8181, 0)	-1
  (8183, 0)	1
  (8184, 0)	1
  (8185, 0)	1
  (8187, 0)	1
  (8189, 0)	1
  (8190, 0)	1
  (8191, 0)	1
  (8192, 0)	1
  (8193, 0)	1
  :	:
  (9651, 208)	-1
  (9662, 208)	-1
  (9684, 208)	-1
  (9712, 208)	-1
  (9720, 208)	-1
  (9765, 208)	-1
  (9780, 208)	-1
  (9782, 208)	-1
  (9787, 208)	-1
  (9790, 208)	-1
  (9805, 208)	-1
  (9835, 208)	-1
  (9854, 208)	-1
  (9861, 208)	-1
  (9884, 208)	-1
  (9919, 208)	-1
  (9924, 208)	-1
  (9941, 208)	-1
  (9971, 208)	-1
  (10013, 208)	-1
  (10070, 208)	-1
  (10079, 208)	-1
  (10162, 208)	-1
  (10196, 208)	-1
  (10489, 208)	-1


In [31]:
labels = the_matrix.toarray()
print(labels.shape)

(10574, 209)


In [32]:
print(compound_index_featuers.shape)

(47139, 633)


In [33]:
compound_index_featuers_train_3 = compound_index_featuers[compound_index_featuers['INCHIKEY'].isin(train_df_3['INCHIKEY'])]
print(compound_index_featuers_train_3.shape)

(32986, 633)


In [34]:
compound_index_featuers_test_3 = compound_index_featuers[compound_index_featuers['INCHIKEY'].isin(test_df_3['INCHIKEY'])]
print(compound_index_featuers_test_3.shape)

(9410, 633)


In [35]:
compound_index_featuers_val_3 = compound_index_featuers[compound_index_featuers['INCHIKEY'].isin(val_df_3['INCHIKEY'])]
print(compound_index_featuers_val_3.shape)

(4743, 633)


In [36]:
data_train_3 = compound_index_featuers_train_3.to_numpy()
print(data_train_3.shape)

(32986, 633)


In [37]:
data_test_3 = compound_index_featuers_test_3.to_numpy()
print(data_test_3.shape)

(9410, 633)


In [38]:
data_val_3 = compound_index_featuers_val_3.to_numpy()
print(data_val_3.shape)

(4743, 633)


In [39]:
print(labels.shape)

(10574, 209)


In [40]:
complete_labels_train_3 =[]
for row in data_train_3:
        complete_labels_train_3.append(labels[row[0]])
complete_labels_train_3 = np.array(complete_labels_train_3)
print(complete_labels_train_3.shape)

(32986, 209)


In [41]:
complete_labels_test_3 =[]
for row in data_test_3:
        complete_labels_test_3.append(labels[row[0]])
complete_labels_test_3 = np.array(complete_labels_test_3)
print(complete_labels_test_3.shape)

(9410, 209)


In [42]:
complete_labels_val_3 =[]
for row in data_val_3:
        complete_labels_val_3.append(labels[row[0]])
complete_labels_val_3 = np.array(complete_labels_val_3)
print(complete_labels_val_3.shape)

(4743, 209)


In [43]:
print(data_train_3)
    
    

[[0 'IENZQIKPVFGBNW-UHFFFAOYSA-N' -0.92419 ... 0.58602 0.88418 0.1362]
 [0 'IENZQIKPVFGBNW-UHFFFAOYSA-N' 6.4768 ... 3.0646 5.2271 6.5882]
 [0 'IENZQIKPVFGBNW-UHFFFAOYSA-N' 0.61509 ... -0.33624 -0.066628 -0.94127]
 ...
 [10571 'YAJYINBQFXCAPI-PFPZSTESSA-N' -1.0928 ... -0.75238 0.69384
  0.13893]
 [10571 'YAJYINBQFXCAPI-PFPZSTESSA-N' -0.83083 ... 0.48064 0.090722
  0.5562]
 [10571 'YAJYINBQFXCAPI-PFPZSTESSA-N' -0.4139 ... 0.6106 -0.2677 0.4296]]


In [44]:
final_featuers_train_3 = data_train_3[:,2:]
print(final_featuers_train_3.shape)

(32986, 631)


In [45]:
final_featuers_test_3 = data_test_3[:,2:]
print(final_featuers_test_3.shape)

(9410, 631)


In [46]:
final_featuers_val_3 = data_val_3[:,2:]
print(final_featuers_val_3.shape)

(4743, 631)


In [47]:
np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/impute_features_train3.npy', final_featuers_train_3)
np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/all_labels_train3.npy', complete_labels_train_3)


np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/impute_features_test3.npy', final_featuers_test_3)
np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/all_labels_test3.npy', complete_labels_test_3)

np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/impute_features_val3.npy', final_featuers_val_3)
np.save('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/all_labels_val3.npy', complete_labels_val_3)


l = np.load('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/all_labels_val3.npy')
f = np.load('/SSL_data/supervisedCNNs/Bray_bioactives/processed_data/data_split_3/impute_features_val3.npy', allow_pickle=True)

print(l == complete_labels_val_3)
print(f == final_featuers_val_3)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
